### About this file
- Load train_data and plug into Huggingface libraries
- Casual Language Modeling using distillgpt2
 1. load texts and concatenate them after tokenization.
 2. split them in examples of sequence length



In [2]:
# install transformers
#! pip install transformers

# import libraries
import numpy as np
import pandas as pd
import transformers
import torch

In [3]:
# load train_data as datasets
# read text file put in DataFrame
datasets = pd.read_fwf('data/train_data.txt', header = None)
df_train = pd.DataFrame(datasets)

# drop all except conversations
# save convo by row 
# row1 - convo 1
# row2 - convo 2
df_train = df_train.iloc[:,0]


# sneak peak
print(df_train.describe())
df_train.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'data/train_data.txt'

In [ ]:
# load distilgpt2 model
model_name = 'distilgpt2'

# load tokenizer
# to tokenize all texts with same vocabs
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# After cleaning data, instantiate Trainer
# Use model distilGPT2
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_name)

# prepare dataset

# put train data into list
texts = df_train.tolist()
text_data = " ".join(texts) #!
# tokenize concatenated text data
input_ids = tokenizer(text_data).input_ids

# create dataset
from transformers import TextDataset
dataset = TextDataset(
    tokenizer = tokenizer,
    text_column = input_ids, # tokenized input_ids #!
    block_size = 128  #adjust block size after discussion
)


NameError: name 'df_train' is not defined

In [ ]:
# Create Trainer instance 
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    f"{model}-Finetuned-train-data",
    evaluation_strategy='epoch',
    laerning_rate=2e-5, #TBD
    weight_decay= .01, #TBD
    push_to_hub=True,
    
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)
# Train data
trainer.train()
